<a href="https://colab.research.google.com/github/SonyaBurg/predict_fb_reactions/blob/main/Prediction_of_reactions_to_the_posts_Test_Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

import re
import string

In [2]:
data = pd.read_csv('fb_sentiment.csv').drop('Unnamed: 0', axis=1)

### The dataset is imbalanced and the texts recquire cleaning and tokenization

In [3]:
def clean_text(text):
 text = text.lower()
 text = re.sub(r'(?P<url>https?://[^\s]+)', "__URL__", text)
 text = re.sub(r'[^\w\s]', '', text)
 regex  = re.compile('\*+')
 text = re.sub(regex, "*", text)
 text = re.sub(r'\w*\d\w*', ' ', text)
 text = ' '.join(text.split())
 return text

In [4]:
data['processed_posts'] = pd.DataFrame(data.FBPost.apply(clean_text))

In [5]:
data

,FBPost,Label,processed_posts
0,Drug Runners and a U.S. Senator have somethin...,O,drug runners and a us senator have something t...
1,"Heres a single, to add, to Kindle. Just read t...",O,heres a single to add to kindle just read this...
2,If you tire of Non-Fiction.. Check out http://...,O,if you tire of nonfiction check out __URL__
3,Ghost of Round Island is supposedly nonfiction.,O,ghost of round island is supposedly nonfiction
4,Why is Barnes and Nobles version of the Kindle...,N,why is barnes and nobles version of the kindle...
...,...,...,...
995,I liked it. Its youth oriented and I think th...,P,i liked it its youth oriented and i think this...
996,"I think the point of the commercial is that, e...",P,i think the point of the commercial is that ev...
997,Kindle 3 is such a great product. I could not ...,P,kindle is such a great product i could not be ...
998,develop a way to share books! that is a big d...,N,develop a way to share books that is a big dra...


In [6]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [7]:
def second_round(text):
  text = ' '.join(word_tokenize(text))
  return ' '.join([i for i in text.split() if i not in stopwords.words('english')])

In [8]:
data['processed_posts'] = data.processed_posts.apply(second_round)

In [10]:
data

,FBPost,Label,processed_posts
0,Drug Runners and a U.S. Senator have somethin...,O,drug runners us senator something murder __URL...
1,"Heres a single, to add, to Kindle. Just read t...",O,heres single add kindle read century story gho...
2,If you tire of Non-Fiction.. Check out http://...,O,tire nonfiction check __URL__
3,Ghost of Round Island is supposedly nonfiction.,O,ghost round island supposedly nonfiction
4,Why is Barnes and Nobles version of the Kindle...,N,barnes nobles version kindle much expensive ki...
...,...,...,...
995,I liked it. Its youth oriented and I think th...,P,liked youth oriented think widen appeal
996,"I think the point of the commercial is that, e...",P,think point commercial even borders closing ma...
997,Kindle 3 is such a great product. I could not ...,P,kindle great product could happier mine
998,develop a way to share books! that is a big d...,N,develop way share books big drawback love kind...


In [11]:
tf_idf = TfidfVectorizer(ngram_range=(1, 2))
text_matrix = tf_idf.fit_transform(data.processed_posts)

In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score

In [15]:
classes = {'P': 0, 'N': 1, 'O': 2}
data['Label'] = data.Label.apply(lambda x: classes[x])

In [16]:
X_train, X_test, y_train, y_test = train_test_split(text_matrix, data['Label'], test_size=0.3)

In [17]:
clf = RandomForestClassifier()
cv = GridSearchCV(clf, {'n_estimators': range(5, 21), 'criterion': ['gini', 'entropy', 'log_loss'], 'max_depth': [None] + [i for i in range(2, 11)]})
cv.fit(X_train, y_train)

GridSearchCV(estimator=RandomForestClassifier(),
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_depth': [None, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                         'n_estimators': range(5, 21)})

In [18]:
cv.best_params_

{'criterion': 'log_loss', 'max_depth': None, 'n_estimators': 19}

In [19]:
clf = RandomForestClassifier(criterion=cv.best_params_['criterion'], n_estimators=cv.best_params_['n_estimators'])
clf.fit(X_train, y_train)

RandomForestClassifier(criterion='log_loss', n_estimators=19)

In [20]:
y_pred = clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.75